In [1]:
from quantdev.data import Databank
from quantdev.backtest import *
from quantdev.analysis import *
import os

import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='space')

In [2]:
path = '/Users/jianrui/Desktop/Career/2025_Dream Jobs/UC Capital 金融交易員(量化交易:高頻交易)'
db = Databank()

#### Classic

In [4]:
ROE = db['常續ROE'].to_rank()
MTM = (db['收盤價']*db['調整係數']).pct_change(60).to_rank()
PBR = (-1*db['股價淨值比']).to_rank()
classic = backtesting(
    (ROE+PBR+MTM).to_rank()>=0.99,
    rebalance='QR',
    start='2008-01-01',
    report=True,
)

,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),25.67,8.80,14.81
Sharpe,1.19,0.56,0.97
Calmar,0.43,0.16,0.58
MDD(%),-59.22,-53.64,-25.74
Simple MDD(%),-64.72,-65.22,-22.51
Win Rate(%),57.61,54.74,53.55
Weekly Win(%),59.93,55.96,54.97
Monthly Win(%),66.03,61.72,55.98
Yearly Win(%),66.67,66.67,77.78
Win/Loss Ratio,0.91,0.92,1.02


In [19]:
def metrics_to_excel(report:Report, name:str):
    report\
        .metrics\
        .loc[['CAGR(%)', 'Sharpe', 'Calmar', 'MDD(%)', 'Monthly Win(%)', 'Yearly Win(%)', 't']]\
        .to_excel(os.path.join(path, f'{name}_metrics.xlsx'))

def equity_curve_to_excel(report:Report, name:str):
    def calc_dd(ec):
        return (ec+1 - (ec+1).cummax()) / (ec+1).cummax()

    pd.concat([
        (1+report.daily_return.rename('daily return')).cumprod()-1,
        (1+report.exp_returns[report.benchmark_id].rename('benchmark')).cumprod()-1,
        calc_dd((1+report.daily_return.rename('daily return')).cumprod()-1).rename('daily return dd'),
        calc_dd((1+report.exp_returns[report.benchmark_id].rename('benchmark')).cumprod()-1).rename('benchmark dd'),
        report.portfolio_df.apply(lambda row: np.count_nonzero(row), axis=1).rename('p_size'),

    ], axis=1).to_excel(os.path.join(path, f'{name}_equity_curve.xlsx'))

#### Profit

In [8]:
profit_factors = {
    'ROE': data['常續ROE'],
    'ROE_Q': data['常續ROE_qoq_diff'],
    'ROE_Y': data['常續ROE_yoy_diff'],
}

profit_factors_analysis = {
    k: factor_analysis(v, rebalance='QR') for k, v in profit_factors.items()
}

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),0.78,6.84,9.35,6.05,7.10,13.28,13.20,13.82,15.70,17.12,16.13
Sharpe,0.13,0.47,0.61,0.42,0.49,0.83,0.82,0.85,0.92,0.93,1.36
Calmar,0.01,0.10,0.15,0.09,0.11,0.22,0.21,0.22,0.25,0.26,0.65
MDD(%),-76.34,-70.88,-63.04,-65.91,-64.79,-61.52,-62.83,-63.52,-63.69,-65.12,-24.87
Simple MDD(%),-94.47,-68.41,-53.89,-81.10,-73.66,-45.19,-50.46,-50.59,-48.10,-43.10,-17.27
Win Rate(%),55.52,57.35,58.23,57.45,58.01,59.12,59.38,58.37,58.55,57.41,53.27
Weekly Win(%),52.79,57.14,58.85,59.22,59.51,60.64,61.02,60.26,59.98,59.41,56.20
Monthly Win(%),50.41,54.51,55.33,58.61,59.84,61.07,61.48,66.80,63.93,63.93,61.89
Yearly Win(%),47.62,61.90,66.67,61.90,66.67,76.19,71.43,71.43,71.43,76.19,90.48
Win/Loss Ratio,0.79,0.78,0.77,0.77,0.76,0.78,0.77,0.80,0.81,0.85,1.09


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),-3.23,5.39,4.37,7.17,9.51,12.71,13.39,14.87,16.62,17.93,20.80
Sharpe,-0.08,0.38,0.33,0.51,0.65,0.80,0.84,0.89,0.95,1.01,1.60
Calmar,-0.04,0.07,0.06,0.11,0.15,0.22,0.21,0.25,0.27,0.28,0.86
MDD(%),-79.64,-72.38,-68.62,-64.52,-61.48,-58.72,-62.97,-59.35,-61.90,-64.30,-24.28
Simple MDD(%),-119.22,-60.90,-81.72,-67.62,-62.57,-45.89,-56.40,-48.69,-55.65,-48.35,-25.04
Win Rate(%),52.87,54.02,54.64,54.36,55.12,55.84,55.60,56.20,55.98,55.94,52.89
Weekly Win(%),50.33,53.26,54.59,55.63,56.20,57.62,57.43,57.62,57.43,56.39,57.43
Monthly Win(%),48.36,54.10,53.28,56.56,57.38,56.97,61.48,60.25,59.02,59.02,63.52
Yearly Win(%),47.62,61.90,61.90,66.67,66.67,66.67,66.67,71.43,66.67,66.67,90.48
Win/Loss Ratio,0.75,0.78,0.75,0.79,0.78,0.79,0.80,0.78,0.80,0.81,1.14


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),2.55,5.55,8.65,9.13,9.98,13.66,14.87,14.74,17.42,16.15,13.15
Sharpe,0.24,0.41,0.58,0.61,0.67,0.87,0.91,0.90,1.01,1.00,2.08
Calmar,0.04,0.08,0.14,0.15,0.16,0.23,0.23,0.21,0.27,0.22,0.62
MDD(%),-68.24,-67.94,-63.52,-62.78,-63.75,-60.56,-63.54,-68.82,-64.71,-72.57,-21.24
Simple MDD(%),-63.91,-60.59,-50.63,-51.22,-52.78,-45.35,-46.55,-56.95,-47.29,-64.52,-19.22
Win Rate(%),55.12,56.08,56.75,57.31,56.99,58.01,57.51,57.89,58.15,57.37,53.57
Weekly Win(%),53.83,55.16,57.33,58.66,58.18,59.04,59.22,58.47,60.17,57.43,58.09
Monthly Win(%),50.82,54.92,57.79,58.20,56.56,57.79,60.66,59.43,61.48,58.20,68.44
Yearly Win(%),61.90,66.67,71.43,71.43,71.43,71.43,71.43,76.19,71.43,71.43,90.48
Win/Loss Ratio,0.77,0.77,0.77,0.76,0.78,0.77,0.79,0.78,0.78,0.81,1.12


In [27]:
profit_factors_analysis[k].summary_dfs['ic'] #['styles'].rename(columns={'Quantiles Returns':k})

,Information Coefficient
mean,0.0869
std,0.0996
IR,87.31%
positive ratio,82.50%


In [28]:
profit_ics = pd.concat([
    profit_factors_analysis[k].summary_dfs['ic'].rename(columns={'Information Coefficient':k}) for k in profit_factors.keys()
], axis=1)
profit_ics
profit_ics.to_excel(os.path.join(path, 'profit_ics.xlsx'))

#### Value

In [107]:
value_factors = {
    'PER':get_factor(-1*data['本益比']),
    'PEG':get_factor(-1*data['每股盈餘_YoY']/data['收盤價']),
    'PBR':get_factor(-1*data['股價淨值比']),
    'MCAP_TO_REV':get_factor(-1*data['個股市值(元)']/data['營業收入']),
}

value_factors_analysis = {
    k: factor_analysis(v, rebalance='QR', start='2008') for k, v in value_factors.items()
}

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),8.79,7.77,8.22,10.27,11.31,11.89,13.17,12.80,12.23,7.72,-1.40
Sharpe,0.56,0.51,0.54,0.66,0.72,0.76,0.84,0.82,0.77,0.53,-0.18
Calmar,0.15,0.13,0.15,0.18,0.20,0.21,0.24,0.22,0.21,0.12,-0.03
MDD(%),-59.32,-59.64,-56.50,-55.81,-56.56,-56.19,-54.49,-59.21,-59.00,-61.94,-48.98
Simple MDD(%),-74.47,-75.54,-67.55,-67.69,-71.14,-69.70,-66.27,-76.83,-73.75,-78.69,-52.98
Win Rate(%),59.39,59.18,58.97,59.08,59.65,59.37,59.86,60.05,59.89,58.90,47.92
Weekly Win(%),60.27,59.82,58.82,60.16,61.15,60.27,60.71,59.82,60.27,58.71,45.28
Monthly Win(%),58.65,57.21,59.62,61.06,60.10,62.50,64.90,63.94,65.87,60.10,48.56
Yearly Win(%),66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,44.44
Win/Loss Ratio,0.76,0.76,0.77,0.78,0.78,0.79,0.79,0.78,0.78,0.77,1.05


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),14.53,13.48,13.24,10.27,8.08,5.91,4.85,4.94,5.18,9.34,-4.71
Sharpe,0.88,0.82,0.81,0.66,0.55,0.43,0.36,0.37,0.38,0.61,-0.87
Calmar,0.27,0.24,0.24,0.18,0.14,0.11,0.08,0.08,0.08,0.15,-0.08
MDD(%),-54.37,-55.92,-56.27,-56.14,-57.00,-56.13,-60.63,-61.19,-61.67,-62.01,-57.96
Simple MDD(%),-61.94,-67.83,-67.74,-72.53,-72.45,-69.40,-80.72,-77.10,-80.73,-80.59,-83.89
Win Rate(%),59.72,60.31,59.51,58.64,58.07,57.83,58.78,58.52,58.61,59.01,47.48
Weekly Win(%),61.82,61.93,60.71,59.49,59.16,58.05,58.27,58.49,57.60,59.05,44.51
Monthly Win(%),62.98,63.94,64.42,62.02,61.06,57.69,54.81,57.69,56.73,59.62,37.50
Yearly Win(%),72.22,66.67,66.67,66.67,72.22,66.67,66.67,66.67,66.67,66.67,22.22
Win/Loss Ratio,0.80,0.77,0.79,0.80,0.80,0.79,0.75,0.76,0.76,0.78,0.96


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),4.09,6.51,8.40,9.08,9.36,11.15,11.94,12.21,15.14,16.47,11.25
Sharpe,0.31,0.45,0.56,0.60,0.62,0.72,0.77,0.78,0.93,0.99,1.24
Calmar,0.07,0.11,0.15,0.15,0.16,0.19,0.21,0.21,0.26,0.26,0.59
MDD(%),-59.08,-58.07,-57.33,-59.62,-58.64,-58.25,-55.96,-56.96,-58.74,-63.40,-18.94
Simple MDD(%),-78.03,-75.29,-75.14,-75.79,-75.81,-68.24,-64.38,-71.83,-67.95,-83.91,-18.02
Win Rate(%),57.46,59.06,59.23,59.82,59.58,59.49,59.96,59.56,59.91,59.65,53.07
Weekly Win(%),57.82,59.38,60.04,59.16,60.27,60.27,60.38,61.82,61.04,60.71,52.72
Monthly Win(%),54.81,59.13,58.17,61.54,60.58,62.50,61.54,61.54,63.46,62.02,59.62
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,72.22,66.67,66.67,66.67,66.67,77.78
Win/Loss Ratio,0.79,0.75,0.77,0.75,0.76,0.78,0.78,0.79,0.80,0.82,1.10


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),6.09,8.66,8.07,9.26,9.27,11.43,11.86,12.25,13.45,15.90,8.93
Sharpe,0.43,0.57,0.54,0.60,0.60,0.73,0.75,0.77,0.83,0.97,1.41
Calmar,0.10,0.15,0.13,0.16,0.16,0.20,0.21,0.21,0.23,0.27,0.84
MDD(%),-60.40,-57.05,-61.80,-57.65,-57.28,-55.95,-57.82,-59.01,-59.00,-58.17,-10.59
Simple MDD(%),-76.37,-70.83,-80.92,-73.15,-72.49,-69.65,-73.51,-75.27,-74.32,-72.06,-9.52
Win Rate(%),57.95,58.75,59.25,59.06,59.65,60.19,59.93,60.24,60.24,60.41,52.93
Weekly Win(%),58.16,59.38,59.49,58.71,59.93,59.71,60.71,60.49,60.38,61.38,54.83
Monthly Win(%),59.13,60.10,60.10,59.62,62.98,61.54,62.50,62.50,61.54,64.90,62.50
Yearly Win(%),66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,88.89
Win/Loss Ratio,0.79,0.78,0.76,0.78,0.76,0.76,0.77,0.77,0.77,0.79,1.13


In [108]:
pd.concat([
    value_factors_analysis[k].summary_dfs['summary_metrics']['ls'].rename(k) for k in value_factors_analysis.keys()
], axis=1)\
# .to_excel(os.path.join(path, 'value_summaries.xlsx'))


,PER,PEG,PBR,MCAP_TO_REV
CAGR(%),-1.40,-4.71,11.25,8.93
Sharpe,-0.18,-0.87,1.24,1.41
Calmar,-0.03,-0.08,0.59,0.84
MDD(%),-48.98,-57.96,-18.94,-10.59
Simple MDD(%),-52.98,-83.89,-18.02,-9.52
Win Rate(%),47.92,47.48,53.07,52.93
Weekly Win(%),45.28,44.51,52.72,54.83
Monthly Win(%),48.56,37.50,59.62,62.50
Yearly Win(%),44.44,22.22,77.78,88.89
Win/Loss Ratio,1.05,0.96,1.10,1.13


In [113]:
pd.concat([
    value_factors_analysis[k].summary_dfs['ic'].rename(columns={'Information Coefficient':k}) for k in value_factors_analysis.keys()
], axis=1)\
.to_excel(os.path.join(path, 'value_ics.xlsx'))



In [115]:
pd.concat([
    value_factors_analysis[k].summary_dfs['quantiles'].rename(columns={'Quantiles Returns':k}) for k in value_factors_analysis.keys()
], axis=1)\
.to_excel(os.path.join(path, 'value_quantiles.xlsx'))

In [123]:
pd.concat([
    value_factors_analysis[k].summary_dfs['summary_metrics']['ls'].rename(k) for k in value_factors_analysis.keys()
], axis=1)

,PER,PEG,PBR,MCAP_TO_REV
CAGR(%),-1.40,-4.71,11.25,8.93
Sharpe,-0.18,-0.87,1.24,1.41
Calmar,-0.03,-0.08,0.59,0.84
MDD(%),-48.98,-57.96,-18.94,-10.59
Simple MDD(%),-52.98,-83.89,-18.02,-9.52
Win Rate(%),47.92,47.48,53.07,52.93
Weekly Win(%),45.28,44.51,52.72,54.83
Monthly Win(%),48.56,37.50,59.62,62.50
Yearly Win(%),44.44,22.22,77.78,88.89
Win/Loss Ratio,1.05,0.96,1.10,1.13


In [191]:
returns = data['報酬率']
turnover = data['周轉率']
days = 20
methods = ['EW', 'HR', 'HR-decay', 'IC', 'IR', 'MAX_IC', 'MAX_IR']
pd.concat([calc_metrics(fast_backtesting(get_rank(combine_factors([
    get_factor(returns.apply(lambda x: ((x>0).rolling(days).sum()))),
    get_factor(-1*turnover.rolling(days).mean())
], rebalance='MR', method=i)) >= 0.5, start='2019')).rename(columns={'Strategy':i}) for i in methods], axis=1)


,EW,HR,HR-decay,IC,IR,MAX_IC,MAX_IR
CAGR(%),17.94,0.0,0.0,0.0,0.0,0.0,0.0
Sharpe,1.44,NaN,NaN,NaN,NaN,NaN,NaN
Calmar,0.70,NaN,NaN,NaN,NaN,NaN,NaN
MDD(%),-25.78,0.0,0.0,0.0,0.0,0.0,0.0
Simple MDD(%),-24.72,0.0,0.0,0.0,0.0,0.0,0.0
Win Rate(%),64.69,0.0,0.0,0.0,0.0,0.0,0.0
Weekly Win(%),64.83,0.0,0.0,0.0,0.0,0.0,0.0
Monthly Win(%),69.74,0.0,0.0,0.0,0.0,0.0,0.0
Yearly Win(%),85.71,0.0,0.0,0.0,0.0,0.0,0.0
Win/Loss Ratio,0.74,NaN,NaN,NaN,NaN,NaN,NaN


#### Momentum

In [195]:
returns = db['報酬率']
turnover = db['周轉率']

days = 20
FIP = get_rank(
    get_factor(
        returns.apply(lambda x: ((x>0).rolling(days).sum()))
    ) - 
    get_factor(turnover.rolling(days).mean())
)

mtm_factors = {
    'MTM_1M':get_factor(db['MTM_1M']),
    'MTM_3M':get_factor(db['MTM_3M']),
    'MTM_1Y':get_factor(db['MTM_1Y']),
    'FIP':get_factor(returns.apply(lambda x: ((x>0).rolling(60).sum()-(x<0).rolling(60).sum())) * (data['MTM_3M'])),
    'FIP_TW':FIP,
}

mtm_factors_analysis = {
    k: factor_analysis(v, rebalance='QR', start='2008') for k, v in mtm_factors.items()
}

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),5.71,5.50,8.17,8.88,10.78,10.39,9.40,10.46,13.67,12.98,6.05
Sharpe,0.38,0.38,0.54,0.60,0.74,0.72,0.66,0.72,0.86,0.75,0.65
Calmar,0.09,0.09,0.14,0.16,0.21,0.18,0.16,0.18,0.23,0.20,0.15
MDD(%),-61.15,-61.85,-57.40,-54.21,-52.00,-57.81,-60.34,-57.74,-59.87,-64.45,-40.77
Simple MDD(%),-86.86,-82.01,-72.54,-64.69,-58.09,-71.35,-73.74,-70.75,-73.51,-86.73,-43.41
Win Rate(%),57.43,58.59,59.46,58.75,59.72,60.71,60.03,60.17,59.89,59.77,53.26
Weekly Win(%),56.38,56.27,58.82,59.38,61.38,62.60,60.60,61.38,62.71,59.71,57.05
Monthly Win(%),58.17,55.77,58.65,62.02,63.94,61.54,62.98,63.46,65.87,62.02,63.46
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,66.67,72.22,66.67,66.67,66.67,66.67
Win/Loss Ratio,0.80,0.76,0.76,0.79,0.78,0.75,0.76,0.76,0.79,0.78,0.99


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),5.86,4.98,6.48,7.40,9.40,9.83,11.99,13.24,13.68,13.38,6.45
Sharpe,0.39,0.36,0.45,0.53,0.66,0.69,0.81,0.86,0.84,0.75,0.64
Calmar,0.09,0.08,0.11,0.13,0.16,0.17,0.21,0.23,0.24,0.24,0.15
MDD(%),-64.57,-62.87,-58.52,-58.24,-58.50,-57.77,-57.60,-56.59,-57.74,-55.52,-42.94
Simple MDD(%),-88.92,-86.79,-74.56,-74.81,-72.75,-72.57,-69.29,-66.20,-70.37,-64.86,-41.31
Win Rate(%),57.43,57.95,58.68,59.18,59.67,60.34,60.67,60.59,60.59,59.96,53.61
Weekly Win(%),55.38,57.38,57.16,60.49,59.93,60.71,61.38,62.60,63.15,60.38,54.94
Monthly Win(%),56.73,53.85,59.13,60.10,62.98,64.42,65.87,65.87,64.42,62.02,62.02
Yearly Win(%),61.11,61.11,66.67,66.67,66.67,72.22,77.78,66.67,72.22,66.67,72.22
Win/Loss Ratio,0.80,0.78,0.77,0.77,0.77,0.76,0.76,0.77,0.76,0.77,0.97


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),6.46,6.93,8.99,9.14,9.80,11.23,12.58,11.35,13.36,10.85,3.77
Sharpe,0.43,0.47,0.60,0.63,0.68,0.77,0.83,0.74,0.82,0.63,0.41
Calmar,0.09,0.11,0.15,0.15,0.17,0.20,0.23,0.21,0.25,0.20,0.07
MDD(%),-68.85,-63.46,-60.86,-59.19,-56.90,-56.45,-54.66,-54.86,-53.51,-55.41,-51.23
Simple MDD(%),-107.88,-83.83,-76.31,-73.69,-71.50,-69.72,-66.08,-63.99,-62.25,-66.84,-46.83
Win Rate(%),58.64,58.71,60.03,59.84,59.91,59.96,60.43,59.96,60.62,58.90,51.82
Weekly Win(%),56.60,57.05,59.16,61.27,60.60,61.04,62.93,61.27,60.49,60.04,52.61
Monthly Win(%),53.37,57.69,60.10,62.02,62.02,64.42,64.42,63.46,62.98,62.02,58.17
Yearly Win(%),66.67,66.67,66.67,66.67,66.67,77.78,72.22,72.22,66.67,66.67,61.11
Win/Loss Ratio,0.77,0.77,0.75,0.76,0.77,0.78,0.77,0.77,0.76,0.78,1.00


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),8.05,7.80,9.31,8.88,8.75,9.29,10.01,10.52,9.94,12.89,4.43
Sharpe,0.53,0.56,0.66,0.64,0.63,0.64,0.66,0.67,0.63,0.75,0.69
Calmar,0.14,0.13,0.16,0.15,0.15,0.16,0.18,0.18,0.17,0.21,0.34
MDD(%),-59.03,-58.33,-57.51,-57.82,-57.36,-58.68,-56.40,-57.72,-60.21,-60.49,-12.88
Simple MDD(%),-72.78,-74.37,-70.92,-71.72,-71.92,-72.55,-68.27,-71.11,-76.34,-75.48,-10.95
Win Rate(%),59.13,59.06,59.75,59.49,59.82,60.45,59.46,59.72,60.24,59.11,50.90
Weekly Win(%),59.05,60.04,61.27,60.38,61.04,59.71,59.82,60.49,59.71,60.04,51.83
Monthly Win(%),60.58,61.54,62.50,62.50,59.62,59.62,61.54,60.58,60.58,60.58,61.06
Yearly Win(%),66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,61.11
Win/Loss Ratio,0.76,0.77,0.77,0.77,0.76,0.74,0.78,0.77,0.75,0.80,1.10


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,ls
CAGR(%),3.08,9.13,8.86,8.66,9.46,11.26,12.37,12.52,13.73,16.53,10.20
Sharpe,0.25,0.55,0.57,0.58,0.67,0.80,0.90,0.94,1.06,1.36,0.82
Calmar,0.05,0.15,0.15,0.14,0.17,0.20,0.24,0.23,0.26,0.33,0.42
MDD(%),-66.35,-62.35,-57.98,-61.79,-56.16,-55.83,-52.51,-54.05,-52.44,-50.24,-24.23
Simple MDD(%),-89.23,-84.88,-75.19,-79.03,-72.06,-67.51,-61.97,-63.40,-59.78,-58.43,-19.55
Win Rate(%),58.73,59.39,59.63,59.77,59.89,59.56,60.57,60.95,60.24,61.63,48.80
Weekly Win(%),58.38,58.27,58.38,59.82,60.71,61.04,61.60,62.26,61.93,63.26,50.06
Monthly Win(%),54.81,57.21,58.65,60.58,60.10,62.02,64.90,65.87,64.90,69.23,57.21
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,72.22,77.78,66.67,77.78,77.78,83.33
Win/Loss Ratio,0.74,0.76,0.76,0.76,0.76,0.80,0.78,0.77,0.82,0.81,1.22


In [70]:
returns = db['報酬率']
turnover = db['周轉率']

days = 20

mtm_factors = {
    f'FIP_{day}':((returns.apply(lambda x: ((x>0).rolling(day).sum()))).to_factor() - (turnover.rolling(day).mean()).to_factor()).to_rank() for day in [20, 60, 120, 240]
}

mtm_factors_analysis = {
    k: factor_analysis(v, rebalance='QR', start='2008') for k, v in mtm_factors.items()
}

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),3.33,9.30,8.96,8.78,9.55,11.45,12.39,12.81,14.02,16.79,10.07
Sharpe,0.26,0.55,0.57,0.58,0.66,0.79,0.89,0.93,1.07,1.36,0.80
Calmar,0.05,0.15,0.15,0.14,0.17,0.21,0.24,0.24,0.27,0.33,0.42
MDD(%),-66.35,-62.35,-57.98,-61.79,-56.16,-55.83,-52.51,-54.05,-52.44,-50.24,-24.23
Simple MDD(%),-89.23,-84.88,-75.19,-79.03,-72.06,-67.51,-61.97,-63.40,-59.78,-58.43,-19.55
Win Rate(%),58.76,59.39,59.58,59.70,59.93,59.58,60.54,60.97,60.17,61.63,48.78
Weekly Win(%),58.39,58.17,58.28,59.82,60.82,61.04,61.70,62.25,62.03,63.47,50.11
Monthly Win(%),54.55,56.94,58.37,60.29,59.81,61.72,64.59,66.03,65.07,69.38,56.94
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,72.22,77.78,66.67,77.78,77.78,83.33
Win/Loss Ratio,0.74,0.76,0.76,0.76,0.76,0.80,0.78,0.77,0.82,0.81,1.21


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),3.91,7.66,8.03,9.66,10.19,11.39,13.42,12.94,15.20,17.60,10.82
Sharpe,0.29,0.47,0.52,0.63,0.68,0.78,0.93,0.94,1.13,1.35,0.95
Calmar,0.06,0.12,0.13,0.16,0.18,0.20,0.25,0.25,0.30,0.36,0.40
MDD(%),-66.76,-61.97,-60.64,-59.58,-58.15,-57.55,-53.70,-52.80,-50.00,-48.71,-27.02
Simple MDD(%),-92.40,-80.12,-79.34,-76.70,-75.04,-73.68,-63.44,-65.81,-55.28,-53.85,-20.52
Win Rate(%),58.17,59.58,60.15,59.91,60.00,60.03,60.26,60.43,60.52,61.20,50.33
Weekly Win(%),59.16,58.50,59.38,60.38,59.38,61.26,61.04,62.03,62.80,63.69,52.43
Monthly Win(%),56.94,57.89,56.94,61.24,61.72,63.64,61.24,62.20,66.03,67.94,59.81
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,72.22,72.22,77.78,72.22,83.33,77.78
Win/Loss Ratio,0.76,0.74,0.73,0.76,0.76,0.78,0.79,0.79,0.82,0.83,1.18


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),5.54,6.96,8.22,7.78,10.47,12.36,12.60,14.11,15.85,17.01,8.80
Sharpe,0.37,0.45,0.54,0.52,0.69,0.83,0.88,1.00,1.13,1.26,0.82
Calmar,0.08,0.11,0.14,0.13,0.18,0.22,0.23,0.27,0.31,0.35,0.28
MDD(%),-66.39,-64.17,-60.39,-60.76,-59.20,-56.50,-54.74,-52.54,-51.17,-49.29,-31.36
Simple MDD(%),-90.84,-87.23,-77.97,-81.70,-75.22,-71.83,-65.68,-61.72,-60.38,-54.27,-25.01
Win Rate(%),58.48,59.56,59.51,59.02,59.82,60.52,60.92,60.92,60.47,60.99,49.67
Weekly Win(%),58.17,59.49,60.38,58.83,60.38,61.59,61.81,62.25,63.13,62.58,51.77
Monthly Win(%),56.94,57.89,56.46,59.33,62.20,59.81,64.59,61.72,68.90,67.46,58.37
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,66.67,72.22,77.78,72.22,77.78,77.78
Win/Loss Ratio,0.76,0.74,0.76,0.77,0.77,0.77,0.76,0.78,0.82,0.82,1.18


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),3.99,6.48,8.37,9.00,11.32,11.19,14.69,16.29,14.73,16.97,10.68
Sharpe,0.30,0.43,0.54,0.59,0.74,0.75,0.97,1.10,1.04,1.23,1.03
Calmar,0.06,0.11,0.13,0.15,0.19,0.19,0.26,0.32,0.29,0.35,0.46
MDD(%),-65.71,-61.28,-63.58,-60.90,-58.73,-57.90,-55.79,-51.37,-51.07,-48.55,-23.12
Simple MDD(%),-93.41,-79.15,-84.50,-79.55,-76.54,-74.17,-67.77,-57.63,-57.79,-53.79,-17.43
Win Rate(%),58.36,59.44,59.79,59.14,60.19,59.82,60.69,60.83,60.33,60.71,50.14
Weekly Win(%),58.50,58.94,59.82,59.27,60.60,60.93,62.03,63.47,62.36,62.47,52.21
Monthly Win(%),55.50,56.94,59.33,60.77,62.68,61.72,65.55,66.03,65.07,68.90,55.98
Yearly Win(%),61.11,61.11,66.67,66.67,66.67,66.67,72.22,72.22,72.22,77.78,83.33
Win/Loss Ratio,0.76,0.74,0.75,0.78,0.77,0.78,0.79,0.80,0.81,0.82,1.20


In [74]:
(mtm_factors_analysis['FIP_20'].metrics)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,LS
CAGR(%),3.33,9.30,8.96,8.78,9.55,11.45,12.39,12.81,14.02,16.79,10.07
Sharpe,0.26,0.55,0.57,0.58,0.66,0.79,0.89,0.93,1.07,1.36,0.80
Calmar,0.05,0.15,0.15,0.14,0.17,0.21,0.24,0.24,0.27,0.33,0.42
MDD(%),-66.35,-62.35,-57.98,-61.79,-56.16,-55.83,-52.51,-54.05,-52.44,-50.24,-24.23
Simple MDD(%),-89.23,-84.88,-75.19,-79.03,-72.06,-67.51,-61.97,-63.40,-59.78,-58.43,-19.55
Win Rate(%),58.76,59.39,59.58,59.70,59.93,59.58,60.54,60.97,60.17,61.63,48.78
Weekly Win(%),58.39,58.17,58.28,59.82,60.82,61.04,61.70,62.25,62.03,63.47,50.11
Monthly Win(%),54.55,56.94,58.37,60.29,59.81,61.72,64.59,66.03,65.07,69.38,56.94
Yearly Win(%),61.11,66.67,66.67,66.67,66.67,72.22,77.78,66.67,77.78,77.78,83.33
Win/Loss Ratio,0.74,0.76,0.76,0.76,0.76,0.80,0.78,0.77,0.82,0.81,1.21


In [ ]:
pd.concat([
    mtm_factors_analysis[k].summary_dfs['ic'].rename(columns={'Information Coefficient':k}) for k in mtm_factors.keys()
], axis=1)\
# .to_excel(os.path.join(path, 'mtm_ics.xlsx'))

In [197]:
pd.concat([
    mtm_factors_analysis[k].summary_dfs['summary_metrics']['ls'].rename(k) for k in mtm_factors.keys()
], axis=1)\
    # .to_excel(os.path.join(path, 'mtm_summaries.xlsx'))

,MTM_1M,MTM_3M,MTM_1Y,FIP,FIP_TW
CAGR(%),6.05,6.45,3.77,4.43,10.20
Sharpe,0.65,0.64,0.41,0.69,0.82
Calmar,0.15,0.15,0.07,0.34,0.42
MDD(%),-40.77,-42.94,-51.23,-12.88,-24.23
Simple MDD(%),-43.41,-41.31,-46.83,-10.95,-19.55
Win Rate(%),53.26,53.61,51.82,50.90,48.80
Weekly Win(%),57.05,54.94,52.61,51.83,50.06
Monthly Win(%),63.46,62.02,58.17,61.06,57.21
Yearly Win(%),66.67,72.22,61.11,61.11,83.33
Win/Loss Ratio,0.99,0.97,1.00,1.10,1.22


In [200]:
(pd.concat([
    mtm_factors_analysis[k].summary_dfs['quantiles'].rename(columns={'Quantiles Returns':k}) for k in mtm_factors_analysis.keys()
], axis=1)\
.replace('%', '', regex=True)\
.astype(float)/100)\
.to_excel(os.path.join(path, 'mtm_quantiles.xlsx'))

#### Parameters

In [3]:
ROE_Y = (db['常續ROE'] - db['常續ROE_lag4']).to_rank()
MCAP_TO_REV = (db['營業收入']/db['個股市值(元)']).to_rank()

returns = db['報酬率']
turnover = db['周轉率']
days = 20
FIP = (
    returns.apply(lambda x: ((x>0).rolling(days).sum())).to_factor() -
    turnover.rolling(days).mean().to_factor()
).to_rank()

In [54]:
f = (ROE_Y+MCAP_TO_REV+FIP).to_rank()

improved_factors = backtesting(
   f>=0.99, 
   rebalance='QR', 
   start='2008', 
   report=True, 
)
# metrics_to_excel(improved_factors, 'improved_factors')
# equity_curve_to_excel(improved_factors, 'improved_factors')


,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),37.36,8.80,24.44
Sharpe,1.92,0.56,1.44
Calmar,0.86,0.16,0.94
MDD(%),-43.64,-53.64,-25.87
Simple MDD(%),-45.18,-65.22,-12.31
Win Rate(%),57.52,54.74,53.76
Weekly Win(%),60.93,55.96,56.29
Monthly Win(%),67.46,61.72,66.51
Yearly Win(%),83.33,66.67,100.00
Win/Loss Ratio,1.04,0.92,1.09


In [46]:
# db.find('處置')
DatasetsHandler().read_dataset('trading_notes', columns=['date', 'stock_id', '是否為處置股票'])

,date,stock_id,是否為處置股票
0,2005-01-03,0001,
1,2005-01-03,0050,
2,2005-01-03,1101,
3,2005-01-03,1101A,
4,2005-01-03,1102,
...,...,...,...
8971528,2025-05-06,Y1900,
8971529,2025-05-06,Y2500,
8971530,2025-05-06,Y2800,
8971531,2025-05-06,Y5556,


In [65]:
(1+(final.daily_return - final.exp_returns[final.benchmark_id] + db['市值比重']['2330']*final.exp_returns['2330']).loc['2008-01-01':].dropna()).cumprod()-1

t_date
2008-01-02   -0.131419
2008-01-03   -0.057771
2008-01-04   -0.552066
2008-01-07   -0.539925
2008-01-08   -0.481184
                ...   
2025-04-28   -1.000000
2025-04-29   -1.000000
2025-04-30   -1.000000
2025-05-02   -1.000000
2025-05-05   -1.000000
Length: 4258, dtype: float64

In [69]:
calc_metrics((final.daily_return - final.exp_returns[final.benchmark_id] + db['市值比重']['2330']/100*final.exp_returns['2330']).loc['2008-01-01':].dropna())

,Strategy
CAGR(%),40.24
Sharpe,1.59
Calmar,1.14
MDD(%),-35.20
Simple MDD(%),-13.23
Win Rate(%),53.10
Weekly Win(%),55.63
Monthly Win(%),63.16
Yearly Win(%),88.89
Win/Loss Ratio,1.18


In [8]:
lq_df = calc_liquidity(final.portfolio_df)

In [53]:
full_settle = (lq_df['是否全額交割']=='Y')
notice = (lq_df['是否為注意股票']=='Y')
disposal = (lq_df['是否為處置股票']=='Y')
buy_limit = ((lq_df['date']==lq_df['sell_date'])&(lq_df['漲跌停註記'] == '-'))
sell_limit = ((lq_df['date']==lq_df['buy_date'])&(lq_df['漲跌停註記'] == '+'))
low_volume = (lq_df['成交量']<50)
low_money = (lq_df['成交金額']<5e5)
pd.concat([
    lq_df[full_settle].groupby('date')['stock_id'].count().rename(f'全額交割股: {full_settle.mean():.2%}'),
    lq_df[notice].groupby('date')['stock_id'].count().rename(f'注意股: {notice.mean():.2%}'),
    lq_df[disposal].groupby('date')['stock_id'].count().rename(f'處置股: {disposal.mean():.2%}'),
    lq_df[buy_limit].groupby('date')['stock_id'].count().rename(f'買在漲停: {buy_limit.mean():.2%}'),
    lq_df[sell_limit].groupby('date')['stock_id'].count().rename(f'賣在跌停: {sell_limit.mean():.2%}'),
    lq_df[low_money].groupby('date')['stock_id'].count().rename(f'成交量<50萬: {low_money.mean():.2%}'),
    lq_df[low_volume].groupby('date')['stock_id'].count().rename(f'成交量<50張: {low_volume.mean():.2%}'),
], axis=1).fillna(0)\
    .iplot(showlegend=True, colorscale='Spectral')

In [91]:
# close = db['收盤價']
# f = (ROE_Y+MCAP_TO_REV+FIP).to_rank(universe=(close>=close.rolling(60).mean()))

final = backtesting(
   f>=0.99, 
   rebalance='QR', 
   start='2008', 
   report=True,
)
# metrics_to_excel(final, 'final')
# equity_curve_to_excel(final, 'final')


,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),48.98,8.80,34.79
Sharpe,1.75,0.56,1.35
Calmar,1.56,0.16,0.94
MDD(%),-31.44,-53.64,-37.20
Simple MDD(%),-26.69,-65.22,-14.51
Win Rate(%),53.64,54.74,52.18
Weekly Win(%),59.60,55.96,55.41
Monthly Win(%),66.51,61.72,60.29
Yearly Win(%),88.89,66.67,83.33
Win/Loss Ratio,1.15,0.92,1.16


In [80]:
close = db['收盤價']
# f = (ROE_Y+MCAP_TO_REV+FIP).to_rank(universe=(close>=close.rolling(60).mean()))

calc_metrics(pd.concat([backtesting(
   (ROE_Y+MCAP_TO_REV+FIP).to_rank(universe=(close>=close.rolling(day).mean()))>=0.99, 
   rebalance='QR', 
   start='2008', 
).rename(day) for day in [5, 20, 60, 120]], axis=1)).to_excel(os.path.join(path, 'ts_mtm_summaries.xlsx'))

In [93]:
plot_return_heatmap((final.daily_return - final.exp_returns[final.benchmark_id] + db['市值比重']['2330']/100*final.exp_returns['2330']).loc['2008-01-01':].dropna())